In [25]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [26]:
chrome_path = r"C:\Users\Leyla\OneDrive - İKTex LLC\Documents\PYTHON\chromedriver\chromedriver.exe"

options = Options()
options.add_argument("--start-maximized")
options.add_argument(
    "User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36"
)

browser = webdriver.Chrome(service=Service(chrome_path), options=options)
wait = WebDriverWait(browser, 15)

browser.get("https://www.amazon.co.uk/")


In [27]:
# ---------------------------
# Create CSV file and header
# ---------------------------
csv_file = open("amazon_products2.csv", "w", newline="", encoding="utf-8")
writer = csv.writer(csv_file)
writer.writerow(["Page", "Title", "Price"])

18

In [28]:
# 1. Accept Cookies Popup
# ---------------------------
try:
    accept = wait.until(EC.element_to_be_clickable((By.ID, "sp-cc-accept")))
    accept.click()
except:
    pass

In [29]:
# 2. Search for product
# ---------------------------
search_box = wait.until(EC.presence_of_element_located((By.ID, "twotabsearchtextbox")))
search_box.send_keys("laptop under 600")

search_button = wait.until(EC.element_to_be_clickable((By.ID, "nav-search-submit-button")))
search_button.click()


In [30]:
# Scrape multiple pages
# ---------------------------
for page in range(1, 4):  # scrape first 3 pages
    print(f"Scraping page {page}...")
    
    # Wait for products to load
    products = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, '//div[@data-component-type="s-search-result"]'))
    )
    
    # Loop through products
    for product in products:
        # URL and Title
        try:
            link_element = product.find_element(By.XPATH, './/h2/parent::a')
            url = link_element.get_attribute("href")
            title = link_element.text.strip()  # text of the <h2> inside the <a>
        except:
            url = "N/A"
            title = "N/A"
        
        # Price
        try:
            # Try standard Amazon price
            price_whole = product.find_element(By.XPATH, './/span[@class="a-price-whole"]').text
            price_fraction = product.find_element(By.XPATH, './/span[@class="a-price-fraction"]').text
            price = f"{price_whole}.{price_fraction}"
        except:
            # Fallback for single span price like <span class="a-color-base">
            try:
                price = product.find_element(By.XPATH, './/span[@class="a-color-base"]').text
            except:
                price = "N/A"
        
        print(title, price, url)
        writer.writerow([page, title, price, url])
    
    # Click next page
    try:
        next_btn = wait.until(
            EC.element_to_be_clickable((By.XPATH, '//a[contains(@class,"s-pagination-next")]'))
        )
        next_btn.click()
        time.sleep(2)
    except:
        print("No more pages.")
        break

# ---------------------------
# Close CSV and browser
# ---------------------------
csv_file.close()
browser.quit()
print("Scraping completed!")

Scraping page 1...
Lenovo IdeaPad 3 Chromebook Laptop | 15.6 inch Full HD Laptop | Intel Pentium Silver N6000 | 8GB RAM | 128GB eMMC| Chrome OS| Arctic Grey AZN 379.38 https://www.amazon.co.uk/Lenovo-IdeaPad-Chromebook-Laptop-Pentium/dp/B0DQVJR5WJ/ref=sr_1_1?crid=1ZCTPSSJSH2C5&dib=eyJ2IjoiMSJ9.6MD-8-nB4Igb0jHYWi_ftFrmVS6TRYVaN0gqkQR4n-3FUohmgAylxz7qTjeXlNNAgimDR0WwZ1dMuKJjgg0MIufwC8h7C5PMis5wMXSRFshvx95wZ9jqFFlI5Sy2YBOpAyKNFuSA3MnDHnLLHgLeITs8sGLhm9YEzlvneTLmNZ91KPM0s6bEZLl1_yIK2FjphAP3Jy35bQrMPQ-lMl_SSRnNrb3aKkiznE7e_6BFE4Y.j96MYFPKgNuwp4xQhCzcB8J2poDMNPByComlOIx5gOM&dib_tag=se&keywords=laptop+under+600&qid=1763626015&sprefix=laptop+under+600%2Caps%2C207&sr=8-1
Lenovo ThinkPad T470 Business Laptop,14-Inch HD Notebook, Intel Core i5-6200U, 16GB RAM, 512GB SSD, QWERTY Keyboard, Windows 11 Pro (Renewed) N/A https://www.amazon.co.uk/Lenovo-ThinkPad-Business-Notebook-i5-6200U/dp/B0FBW6BL97/ref=sr_1_2?crid=1ZCTPSSJSH2C5&dib=eyJ2IjoiMSJ9.6MD-8-nB4Igb0jHYWi_ftFrmVS6TRYVaN0gqkQR4n-3FUohmgAylxz

In [13]:
# 3. SCRAPE MULTIPLE PAGES
# ---------------------------
page = 1

while True:
    print(f"\n--- SCRAPING PAGE {page} ---")

    # Wait for products
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.s-main-slot > div")))

    products = browser.find_elements(By.CSS_SELECTOR, "div[data-component-type='s-search-result']")

    for product in products:
        # Title
        try:
            title = product.find_element(By.CSS_SELECTOR, "h2 a").text
        except:
            title = "No title"

        # Price
        try:
            price = product.find_element(By.CSS_SELECTOR, "span.a-price > span.a-offscreen").text
        except:
            price = "No price"

        # Save to CSV
        writer.writerow([page, title, price])

        print(title, " | ", price)

    # Pagination
    try:
        next_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.s-pagination-next")))
        browser.execute_script("arguments[0].scrollIntoView(true);", next_btn)
        time.sleep(1)
        next_btn.click()
        page += 1
        time.sleep(2)
    except:
        print("No more pages.")
        break


csv_file.close()
browser.quit()


--- SCRAPING PAGE 1 ---
No title  |  No price
No title  |  
No title  |  No price
No title  |  
No title  |  No price
No title  |  
No title  |  No price
No title  |  No price
No title  |  No price
No title  |  
No title  |  No price
No title  |  
No title  |  
No title  |  No price
No title  |  
No title  |  

--- SCRAPING PAGE 2 ---
No title  |  No price
No title  |  No price
No title  |  No price
No title  |  
No title  |  
No title  |  
No title  |  
No title  |  No price
No title  |  No price
No title  |  
No title  |  
No title  |  No price
No title  |  No price
No title  |  
No title  |  
No title  |  No price

--- SCRAPING PAGE 3 ---
No title  |  
No title  |  
No title  |  
No title  |  No price
No title  |  
No title  |  No price
No title  |  No price
No title  |  
No title  |  No price
No title  |  No price
No title  |  
No title  |  No price
No title  |  
No title  |  
No title  |  No price
No title  |  

--- SCRAPING PAGE 4 ---
No title  |  No price
No title  |  
No title